In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

In [39]:
train_data = pd.read_csv('../data_new/train.csv')
test = pd.read_csv('../data_new/test.csv')

C:\Users\ADE17\AppData\Local\Temp\ipykernel_9532\2619112913.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('../data_new/train.csv')


In [38]:
target = train_data.iloc[:,-29:]
target_cols = train_data.iloc[:,-29:].columns

In [25]:
train_data.Blood_Month_sample = train_data.Blood_Month_sample.fillna(train_data.Blood_Month_sample.mean())
train_data.Blood_Month_sample = train_data.Blood_Month_sample.astype('int64')

In [43]:
mask = train_data[target_cols].apply(lambda x: 9 not in x.values, axis=1)
train = train_data[mask]

In [44]:
new_target = train.iloc[:,-29:]

In [27]:
train = train.drop(target, axis=1)

In [28]:
def multival_onehot_enc(df, col_names):
    for col_name in col_names:
        print(col_name)
        df[col_name] = df[col_name].str.replace('.', ',')

        split_values = df[col_name].str.split(',').explode()
        unique_values = split_values.unique()

        for i, value in enumerate(unique_values):
            df[f"{col_name}_{i}"] = 0
            df[f"{col_name}_{i}"] = df[f"{col_name}_{i}"].astype(int)

        for index, row in df.iterrows():
            values = str(row[col_name]).split(',')
            for value in values:
                for i, unique_value in enumerate(unique_values):
                    if value.strip() == unique_value:
                        df.at[index, f"{col_name}_{i}"] = 1
                        break
        df.drop(col_name, axis=1, inplace= True)
    
    return df

In [29]:
cols = ['Treatment_of_athsma', 'General_cofactors',
        'Treatment_of_atopic_dematitis', 'Age_of_onsets']
cols2 = ['French_Region', 'Chip_Type', 
        'Blood_Month_sample', 'Skin_Symptoms',
        'Treatment_of_rhinitis']
train.Treatment_of_rhinitis = train.Treatment_of_rhinitis.astype(str).str.split(pat='.', expand=True)[0]
train = pd.get_dummies(train, columns=cols2)
new_train = multival_onehot_enc(train, cols)

Treatment_of_athsma
General_cofactors


C:\Users\ADE17\AppData\Local\Temp\ipykernel_9532\809910423.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')
C:\Users\ADE17\AppData\Local\Temp\ipykernel_9532\809910423.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


Treatment_of_atopic_dematitis
Age_of_onsets


C:\Users\ADE17\AppData\Local\Temp\ipykernel_9532\809910423.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')
C:\Users\ADE17\AppData\Local\Temp\ipykernel_9532\809910423.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


In [30]:
df_num = new_train.select_dtypes(include=object)

In [31]:
new_train = new_train.drop(df_num, axis=1)

In [12]:
cols_test = ['Treatment_of_athsma', 'General_cofactors',
        'Treatment_of_atopic_dematitis']
cols2_test = ['French_Region', 'Chip_Type', 
        'Blood_Month_sample', 'Skin_Symptoms',
        'Treatment_of_rhinitis', 'Age_of_onsets']
test.Treatment_of_rhinitis = test.Treatment_of_rhinitis.astype(str).str.split(pat='.', expand=True)[0]
test_df = pd.get_dummies(test, columns=cols2_test)
new_test = multival_onehot_enc(test_df, cols_test)

Treatment_of_athsma
General_cofactors
Treatment_of_atopic_dematitis


C:\Users\ADE17\AppData\Local\Temp\ipykernel_9532\707930443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')
C:\Users\ADE17\AppData\Local\Temp\ipykernel_9532\707930443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')
C:\Users\ADE17\AppData\Local\Temp\ipykernel_9532\707930443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


In [13]:
df_num_test = new_test.select_dtypes(include=object)
new_test = new_test.drop(df_num_test, axis=1)
new_test = new_test.drop('trustii_id', axis=1)

In [14]:
new_test.shape

(586, 401)

In [32]:
new_train.shape

(1356, 411)

In [33]:
df1 = new_train
df2 = new_test
columns_df1 = set(df1.columns)
columns_df2 = set(df2.columns)

different_columns = list(columns_df1 - columns_df2)
different_columns.extend(list(columns_df2 - columns_df1))

In [34]:
different_columns

['French_Region_regionO',
 'General_cofactors_19',
 'French_Region_regionN',
 'Age_of_onsets_8',
 'Treatment_of_atopic_dematitis_8',
 'General_cofactors_20',
 'General_cofactors_18',
 'General_cofactors_17',
 'Age_of_onsets_7',
 'General_cofactors_16']

In [35]:
df1 = new_train
df2 = new_test
common_cols = set(df1.columns).intersection(df2.columns)
X_updated = df1.reindex(columns=common_cols)
test_updated = df2.reindex(columns=common_cols)

In [45]:
print(X_updated.shape)
print(test_updated.shape)
print(new_target.shape)

(1356, 401)
(586, 401)
(1356, 29)


In [46]:
X_updated.to_csv("saved_csv/train_encoded_9s_dropped.csv", index=False)
test_updated.to_csv("saved_csv/test_encoded_9s_dropped.csv", index=False)
new_target.to_csv("saved_csv/target_9s_dropped.csv", index=False)